Name: Ahmad Karami

In this assignment, we will implement and test REINFORCE and PPO, which are both on-policy RL algortihms.

# REINFORCE algorithm **(40 points)**

## Setup

We must first install the required packages.

In [1]:
!pip -q install gymnasium[mujoco]
!pip install imageio -q

In [1]:
import gymnasium as gym
import random
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from collections import namedtuple, deque
import imageio

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions import Categorical
from torch.autograd import Variable
import statistics
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Explore the environment

We will train an REINFORCE agent on the `CartPole` environment.

This code displays a video given it's path.

In [2]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Explore the `CartPole` environment using random actions. At each timestep, render the current frame, and use it to make a video of the trajectory.

In [3]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
frames = []

env.reset()
for _ in range(100):
    frames.append(env.render())
    action = env.action_space.sample() # select a random action
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./CartPole.mp4', frames, fps=25)
show_video('./CartPole.mp4')


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [5]:
# Print the observation space and action space
print('Observations:', env.observation_space)
print('Actions:', env.action_space)

Observations: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Actions: Discrete(2)


## Policy Network **(10 points)**

Complete the following code to build an agent that predicts the the probability of playing each action, given the state.

In [5]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        # Define the Policy Network architecture
        self.layer1= nn.Linear(input_size,hidden_size)
        self.mid0= nn.Linear(hidden_size,hidden_size)
        self.mid1= nn.Linear(hidden_size,hidden_size)
        self.layer3= nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # predict the probability of playing each action
        x = Variable(torch.from_numpy(x).float().unsqueeze(0))
        x =  F.relu(self.layer1(x))
        x =  F.relu(self.mid0(x))
        x = F.relu(self.mid1(x))
        action_probs = F.softmax(self.layer3(x),dim= -1)
        return action_probs

## Agent **(20 points)**

REINFORCE algorithm works by interacting with an environment by taking actions based on a policy. As the agent collects rewards from the environment, it records the outcomes and the **log probabilities** of the actions it took. At the end of an episode, the algorithm calculates the total **discounted reward** from each step—this is known as the return.

$$ R_t = \sum_{k=t}^{T} \gamma^{k-t} r_k
 $$

These returns are used to weight the logged probabilities, actions that lead to higher returns are made more probable.


$$ \theta \leftarrow \theta + \alpha \sum_{t=0}^{T-1} \gamma^t R_t \nabla_\theta \log \pi_\theta(a_t|s_t)
 $$


In [6]:
class REINFORCEAgent:
    def __init__(self, policy, optimizer, gamma=0.99):
        self.policy = policy
        self.optimizer = optimizer
        self.gamma = gamma
        self.log_probs = []
        self.rewards = []

    def select_action(self, state):
        # select an action by sampling from the actor's response
        policy_dist = self.policy(state)
        dist = policy_dist.detach().numpy()
        probs = np.squeeze(dist)
        action = np.random.choice([0, 1], p= probs)

        log_prob = torch.log(policy_dist.squeeze(0)[action])
        self.log_probs.append(log_prob)

        return action

    def update_policy(self):
        R = 0
        policy_loss = []
        returns = []

        # Calculate the discounted reward
        for r in self.rewards[::-1]:
            R = R * self.gamma + r
            returns.insert(0,R)

        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-7)

        # Calculate the policy loss
        for log_prob, disR in zip(self.log_probs, returns):
            policy_loss.append(-log_prob * disR )


        self.optimizer.zero_grad()
        policy_loss = torch.stack(policy_loss)
        policy_loss = policy_loss.mean()
        policy_loss.backward()
        self.optimizer.step()

        # Reset the rewards and log probabilities
        del self.rewards[:]
        del self.log_probs[:]

    def store_reward(self, reward):
        self.rewards.append(reward)


## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [50]:
def Plot(results, lr):
    plt.style.use('ggplot')  # Use a nicer style
    fig, ax = plt.subplots(figsize=(5, 3))  # Increase the figure size

    ax.plot(results, label='Rewards')  # Add a label for the legend
    ax.set_title(f'Learning Rate: {lr:.5f}->Perfects: {results.count(500)}' , fontsize=8)
    ax.set_xlabel(f'Episodes=> Mean: {int(statistics.mean(results))}\nSTD: {int(statistics.variance(results) ** 0.5)}', fontsize=8)
    ax.set_ylabel(f'Reward', fontsize=10)
    plt.show()

In [53]:
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
policy = PolicyNetwork(input_size, 128, output_size)
lr = 8e-4
optimizer = optim.Adam(policy.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=50, gamma=0.99)

In [ ]:
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
lr = 8e-4

policy = PolicyNetwork(input_size, 128, output_size)
optimizer = optim.Adam(policy.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=15, gamma=0.98)
agent = REINFORCEAgent(policy, optimizer)


num_episodes = 1000
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0

    # collect rewards and log probabilities for updating the policy in a loop
    while True:
          action  = agent.select_action(state)
          next_state, reward, done, terminate, _ = env.step(action)
          agent.store_reward(reward)
          state = next_state
          total_reward += reward
          if done or terminate:
              break
    agent.update_policy()
    scheduler.step()
    if episode % 50 == 0:
        print(f'Episode {episode+1}: Total Reward = {total_reward}')
env.close()

Episode 1: Total Reward = 14.0
Episode 51: Total Reward = 20.0
Episode 101: Total Reward = 77.0
Episode 151: Total Reward = 163.0
Episode 201: Total Reward = 414.0
Episode 251: Total Reward = 204.0
Episode 301: Total Reward = 500.0
Episode 351: Total Reward = 439.0
Episode 401: Total Reward = 247.0
Episode 451: Total Reward = 111.0
Episode 501: Total Reward = 283.0
Episode 551: Total Reward = 500.0
Episode 601: Total Reward = 500.0
Episode 651: Total Reward = 500.0
Episode 701: Total Reward = 226.0
Episode 751: Total Reward = 421.0
Episode 801: Total Reward = 500.0
Episode 851: Total Reward = 500.0
Episode 901: Total Reward = 500.0
Episode 951: Total Reward = 500.0


## Evaluation **(5 points)**

Here we use the trained agent and collect a trajectory using it's policy. Calculate the cumulative reward by adding rewards in each time space. Save and display the video of this run in the end.

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# run the policy in the environment in a loop
while True:
    frames.append(env.render())
    action  = agent.select_action(state)
    state, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    if terminated or truncated:
        break

env.close()
print(f'Total Reward: {total_reward}')

imageio.mimsave('./eval_reinforce.mp4', frames, fps=25)
show_video('./eval_reinforce.mp4')

Total Reward: 500.0


# Proximal Policy Optimization **(60 points)**

## Setup

## Explore the environment

This code is essential for rendering MUJOCO based environments.

In [4]:
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

env: MUJOCO_GL=egl


We will train a PPO agent in the `HalfCheetah` environment. This environment features continuous actions and more complex mechanics.

Explore this environment using random actions as well, and display the video of the resulting trajectory.

* What are the observation and action spaces of this environment?

* Are values bounded?

In [ ]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
env.reset()
frames = []

for _ in range(100):
    frames.append(env.render())
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./HalfCheetah.mp4', frames, fps=25)
show_video('./HalfCheetah.mp4')


## Actor & Critic **(15 points)**

Proximal Policy Optimization (PPO) is an advanced reinforcement learning algorithm that uses separate actor and critic networks to optimize policy performance.

The actor network is responsible for predicting a probability distribution over actions (discrete) or estimating the value for each action (continuous), given the current state, while the critic network evaluates how good the action taken by the actor is, by predicting the reward based on state.


In [5]:
class Actor(nn.Module):
    def __init__(self, state_dim, hidden_size, action_dim):
        super(Actor, self).__init__()
        # Define the Actor architecture
        self.layer1= nn.Linear(state_dim, hidden_size)

        # self.mid0= nn.Linear(hidden_size, hidden_size)
        self.mid1= nn.Linear(hidden_size, hidden_size)
        self.mid2= nn.Linear(hidden_size, hidden_size)

        self.mu= nn.Linear(hidden_size, action_dim)
        self.std= nn.Linear(hidden_size, action_dim)

    def forward(self, state):
        # In case of continuous environment, we usually
        # predict a mean and std for each action and sample
        # the action from a normal distribution
        if isinstance(state, np.ndarray):
          state = torch.from_numpy(state).float().unsqueeze(0)
        state = state.to(device)

        X =  F.relu(self.layer1(state.float()))
        # X =  F.relu(self.mid0(X))
        X =  F.relu(self.mid1(X))
        X =  F.relu(self.mid2(X))

        mu =  self.mu(X)
        std = F.softplus(self.std(X))

        return mu, std

class Critic(nn.Module):
    def __init__(self, state_dim, hidden_size):
        super(Critic, self).__init__()
        # Define the Critic architecture
        self.layer1= nn.Linear(state_dim, hidden_size)

        # self.mid0= nn.Linear(hidden_size, hidden_size)
        self.mid1= nn.Linear(hidden_size, hidden_size)
        self.mid2= nn.Linear(hidden_size, hidden_size)
        self.value= nn.Linear(hidden_size, 1)

    def forward(self, state):
        # Predict the value of state
        if isinstance(state, np.ndarray):
          state = torch.from_numpy(state).float().unsqueeze(0)
        state = state.to(device)

        X =  F.relu(self.layer1(state.float()))
        # X =  F.relu(self.mid0(X))
        X =  F.relu(self.mid1(X))
        X =  F.relu(self.mid2(X))

        value =  self.value(X)
        return value


## Memory

PPO algorithms need to store sequences of actions, states, log probabilities, rewards, and state values to train the agent. This data is captured in the `Memory` class, which facilitates batch processing by holding and then clearing these elements at the end of each training iteration.

In [6]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]


## Agent **(35 points)**

In PPO, the actor's goal is to maximize the expected return. However, direct maximization can cause large policy updates, risking instability. To prevent this, PPO employs a clipping mechanism, limiting policy changes to a defined range.

$$ L^{CLIP}(\theta) = \hat{\mathbb{E}}_t \left[ \min(r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t) \right]
 $$

Additionally, it uses a probability ratio to scale updates, ensuring changes This ratio provides a scaling factor for the policy updates, ensuring that changes are made in proportion to the improvement in policy performance.

$$ r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}
 $$

 The critic aims to minimize the error between its predictions and the actual returns.

 $$ L^{VF}(\phi) = \left( V_\phi(s_t) - \hat{R}_t \right)^2
 $$

In [7]:
class PPO(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=64, lr=1e-4, gamma=0.99, epochs=4, eps_clip=0.2):
        super(PPO, self).__init__()
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.epochs = epochs

        self.actor = Actor(state_dim, hidden_size, action_dim).to(device)
        self.critic = Critic(state_dim, hidden_size).to(device)

        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=lr)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=lr)
        self.memory = Memory()

    def select_action(self, state):
        # Save state, action, log probability and state value of current step in the memory buffer.
        # predict the actions by sampling from a normal distribution
        # based on the mean and std calculated by actor
        mean, std = self.actor(state)
        dist_policy = torch.distributions.Normal(mean, std)
        action = dist_policy.sample()
        log_prob = dist_policy.log_prob(action).sum()


        value = self.critic(state)

        state = torch.from_numpy(state).float()

        self.memory.states.append(state)
        self.memory.actions.append(action)
        self.memory.logprobs.append(log_prob)
        self.memory.state_values.append(value)

        action = action.squeeze(0)
        action = action.cpu().numpy()
        return action

    def evaluate(self, state, action):
        # evaluate the state value of this state and log probability of choosing this action
        mean, std = self.actor(state)
        state_value = self.critic(state)
        dist = torch.distributions.Normal(mean, std)
        action_logprobs = dist.log_prob(action.squeeze(1)).sum(1)
        entropy = dist.entropy().sum(1)

        return action_logprobs, state_value, entropy

    def update(self):
        rewards = []
        advantages = []
        discounted_reward = 0
        gae = 0
        gae_lambda = 0.95


        # load saved states, actions, log probs, and state values
        old_states = torch.stack(self.memory.states).detach()
        old_actions = torch.stack(self.memory.actions).detach()
        old_logprobs = torch.stack(self.memory.logprobs).detach()
        old_state_vals = torch.stack(self.memory.state_values).detach()

        # Calculate advantages for each timestep (usually difference of rewards and state values)
        for r, v, nv in zip(reversed(self.memory.rewards), reversed(self.memory.state_values), reversed(self.memory.state_values[1:] + [0])):
            discounted_reward = r + self.gamma * discounted_reward
            rewards.insert(0, discounted_reward)

            delta = r + self.gamma * nv - v
            gae = delta + self.gamma * gae_lambda * gae
            advantages.insert(0, gae)

        rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
        # rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-8)

        advantages = torch.tensor(advantages, dtype=torch.float32, device=device)
        # advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)


        loss_ac = 0
        loss_cri = 0
        for _ in range(self.epochs):
            # calculate logprobs and state values based on the new policy
            logprobs, state_values, entropy = self.evaluate(old_states, old_actions)

            # Calculate the loss function and perform the optimization
            ratios = torch.exp(logprobs - old_logprobs.detach())
            Bound1 = ratios * advantages
            Bound2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            loss_actor = -torch.min(Bound1, Bound2) - 0.01 * entropy.mean()
            loss_critic = F.mse_loss(state_values.squeeze(1), rewards)

            loss_actor = loss_actor.mean()
            loss_critic = loss_critic.mean()


            self.optimizer_actor.zero_grad()
            loss_actor.backward(retain_graph=True)
            loss_ac += loss_actor.item()
            self.optimizer_actor.step()

            self.optimizer_critic.zero_grad()
            loss_critic.backward()
            loss_cri += loss_critic.item()
            self.optimizer_critic.step()
        # clear the buffer
        self.memory.clear()
        return loss_ac, loss_cri
    def store_reward(self, reward):
      self.memory.rewards.append(reward)

## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [8]:
env = gym.make("HalfCheetah-v4")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
hidden_size = 128

lr = 5e-4


agent  = PPO(state_dim, action_dim, hidden_size=hidden_size, lr=lr)

# We need to train for many more steps to achieve acceptable results compared to the last environment
num_episodes = 3000

actor_losses = []
critic_losses = []
moving_rewards = np.array([])
sum_of_rewards = []
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    # write the training loop
    while True:
          action = agent.select_action(state)
          next_state, reward, done, terminate, _ = env.step(action)
          agent.store_reward(reward)
          total_reward += reward
          state = next_state

          if done or terminate:
              break

    loss_ac, loss_cri = agent.update()
    actor_losses.append(loss_ac)
    critic_losses.append(loss_cri)
    moving_rewards = np.append(moving_rewards, total_reward)

    if episode % 100 == 0:
        print(f"actor loss:\t{loss_ac:.6f}")
        print(f"critic loss:\t{loss_cri:.6f}")
        print(f'Episode {episode}: Going Reward = {moving_rewards.mean():.1f}: Std = {moving_rewards.std():.1f}')
        moving_rewards = np.array([])

env.close()


actor loss:	37.275601
critic loss:	12639.419189
Episode 0: Going Reward = -589.2: Std = 0.0
actor loss:	8.710280
critic loss:	778.807053
Episode 100: Going Reward = 3.0: Std = 320.6
actor loss:	15.258806
critic loss:	5793.122925
Episode 200: Going Reward = 251.4: Std = 211.1
actor loss:	-2.211996
critic loss:	3643.450195
Episode 300: Going Reward = 371.6: Std = 441.9
actor loss:	-1.360243
critic loss:	7989.034790
Episode 400: Going Reward = 746.4: Std = 766.8
actor loss:	16.505829
critic loss:	7782.610107
Episode 500: Going Reward = 780.2: Std = 798.5
actor loss:	8.137836
critic loss:	2227.619446
Episode 600: Going Reward = 1153.6: Std = 985.8
actor loss:	3.073104
critic loss:	4492.792969
Episode 700: Going Reward = 945.2: Std = 939.2
actor loss:	-8.678360
critic loss:	2129.824402
Episode 800: Going Reward = 536.4: Std = 455.3
actor loss:	-1.091817
critic loss:	1634.782837
Episode 900: Going Reward = 648.4: Std = 188.3
actor loss:	-5.649519
critic loss:	10640.167969
Episode 1000: Going

## Evaluation **(5 points)**

Evaluate the trained policy on the environment. Calculate the cumulative reward and display the video of the trajectory.

In [9]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# run the policy in the environment in a loop
while True:
    frames.append(env.render())
    action  = agent.select_action(state)
    state, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    if terminated or truncated:
        break

env.close()
print(f'Total Reward: {total_reward}')

imageio.mimsave('./eval_ppo.mp4', frames, fps=25)
show_video('./eval_ppo.mp4')

Total Reward: 3834.04726054387
